# Transcriptional trajectories in mouse limb buds reveal the transition from anterior-posterior to proximal-distal patterning at early limb bud stage.
Desanlis I, Paul R, Kmita M. J Dev Biol. 2020;8(4):1–16. 
 
 PMID: [7768367](http://www.ncbi.nlm.nih.gov/pmc/articles/pmc7768367/)
 ***
 

## 0. Setting up workenvironment<a id="0"></a>

In [ ]:
suppressPackageStartupMessages({
    library(DropletUtils)
    library(SingleCellExperiment)
    library(scuttle)
    library(Seurat)
    library(SeuratWrappers)
    library(stringr)
    library(dplyr)
    library(data.table)
    library(Matrix)
    library(patchwork)
    library(ggplot2)
})

options(repr.plot.width = 16, repr.plot.height = 8)

## 4. E11.5<a id="18"></a>

### 4.1. Data analysis Seurat<a id="19"></a>

In [ ]:
E11_5 <- readRDS(file = "/home/host_home/tom/Kmita/6-kmita/RDS_QC/E11_5.Rds")

In [ ]:
VlnPlot(E11_5, features = c("nCount_RNA", "nFeature_RNA", "subsets_Mito_percent"), 
        ncol = 3, group.by = "orig.ident", pt.size = 0)

#### 4.1.1. Feature selection

We next calculate a subset of features that exhibit high cell-to-cell variation in the dataset (i.e, they are highly expressed in some cells, and lowly expressed in others). We and others have found that focusing on these genes in downstream analysis helps to highlight biological signal in single-cell datasets.

The procedure in Seurat v3 directly models the mean-variance relationship inherent in single-cell data, and is implemented in the `FindVariableFeatures` function. By default, it defaults to 2,000 features (genes) per dataset. These will be used in downstream analysis, like principal component analysis (PCA).

In [ ]:
E11_5 <- NormalizeData(E11_5, verbose = FALSE)
E11_5 <- FindVariableFeatures(E11_5, selection.method = "vst", nfeatures = 2000)

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(E11_5), 10)

In [ ]:
options(repr.plot.width=10)

# plot variable features with and without labels
plot1 <- VariableFeaturePlot(E11_5)
plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)
plot2

#### 4.1.2. Scaling the data

Next, we apply a linear transformation (scaling) that is a standard pre-processing step prior to dimensional reduction techniques like PCA. The ScaleData function:

- Shifts the expression of each gene, so that the mean expression across cells is 0
- Scales the expression of each gene, so that the variance across cells is 1
- This step gives equal weight in downstream analyses, so that highly-expressed genes do not dominate. This is superior to log-normalized counts, which overemphasize the influence of small count fluctuations.
- The results of this are stored in `object[["originalexp"]]@scale.data`

In [ ]:
all.genes <- rownames(E11_5)
E11_5 <- ScaleData(E11_5, features = all.genes)

#### 4.1.3. Linear dimensionality reduction: PCA

Next we perform PCA on the scaled data. By default, only the previously determined variable features are used as input, but can be defined using features argument if you wish to choose a different subset.

In [ ]:
E11_5 <- RunPCA(E11_5, features = VariableFeatures(object = E11_5), verbose = FALSE, npcs = 85)

In [ ]:
DimPlot(E11_5, reduction = "pca", group.by = "Sample_Name") + xlab("PC 1") + ylab("PC 2")

#### 4.1.4. Determine dimensionality

To overcome the extensive technical noise in any single feature for scRNA-seq data, Seurat clusters cells based on their PCA scores, with each PC essentially representing a 'metafeature' that combines information across a correlated feature set. The top principal components therefore represent a robust compression of the dataset. However, how many componenets should we choose to include? 

[Macosko et al](http://www.cell.com/abstract/S0092-8674(15)00549-8) introduced a resampling test inspired by the JackStraw procedure. They randomly permute a subset of the data (1% by default) and rerun PCA, constructing a 'null distribution' of feature scores, and repeat this procedure. 'Significant' PCs are defined as those who have a strong enrichment of low p-value features.

**NOTE: This process can take a long time for big datasets, and is therefore commented out for expediency. More approximate techniques such as a scree plot implemented in `ElbowPlot()` can be used to reduce computation time.**

The `JackStrawPlot` function provides a visualization tool for comparing the distribution of p-values for each PC with a uniform distribution (dashed line). 'Significant' PCs will show a strong enrichment of features with low p-values (solid curve above the dashed line).

In [ ]:
# object <- JackStraw(object, num.replicate = 100)
# object <- ScoreJackStraw(object, dims = 1:20)

In [ ]:
# JackStrawPlot(object, dims = 1:15)

An alternative heuristic method generates an 'Elbow plot': a ranking of principle components based on the percentage of variance explained by each one (ElbowPlot function). 

In [ ]:
ElbowPlot(E11_5, 85)

### 4.2. Clustering<a id="20"></a>

Seurat v4 applies a graph-based clustering approach, building upon initial strategies in [Macosko et al](http://www.cell.com/abstract/S0092-8674(15)00549-8). Importantly, the distance metric which drives the clustering analysis (based on previously identified PCs) remains the same. The new method embeds cells in a graph structure - for example a K-nearest neighbor (KNN) graph, with edges drawn between cells with similar feature expression patterns, and then attempt to partition this graph into highly interconnected 'quasi-cliques' or 'communities'.

As in [PhenoGraph](https://doi.org/10.1016/j.cell.2015.05.047), Seurat first constructs a KNN graph based on the euclidean distance in PCA space, and refine the edge weights between any two cells based on the shared overlap in their local neighborhoods (Jaccard similarity). This step is performed using the `FindNeighbors` function, and takes as input the previously defined dimensionality of the dataset.

To cluster the cells, modularity optimization techniques such as the Louvain algorithm are then applied to iteratively group cells together, with the goal of optimizing the standard modularity function. The `FindClusters` function implements this procedure and contains a resolution parameter that sets the 'granularity' of the downstream clustering, with increased values leading to a greater number of clusters. Optimal resolution often increases for larger datasets.

In [ ]:
E11_5 <- FindNeighbors(E11_5, dims = 1:35)
E11_5 <- FindClusters(E11_5, resolution = 0.095)

### 4.3. Non-linear dimensionality reduction: UMAP<a id="21"></a>

Seurat offers several non-linear dimensional reduction techniques, such as tSNE and UMAP, to visualize and explore these datasets. The goal of these algorithms is to learn the underlying manifold of the data in order to place similar cells together in low-dimensional space. Cells within the graph-based clusters determined above should co-localize on these dimension reduction plots. As input to the UMAP and tSNE, [it is recommended to use the same PCs](https://doi.org/10.1038/s41467-019-13056-x) as input to the clustering analysis.

In [ ]:
E11_5 <- RunUMAP(E11_5, dims = 1:35, n.neighbors = 12, verbose = FALSE)

In [ ]:
options(repr.plot.width = 8)

DimPlot(E11_5, reduction = "umap", label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
#ggsave("./6-kmita/images/E11_5_no_annotated.png" ,width=8, height=8)

### 4.4. Differential expression analysis<a id="22"></a>
#### 4.4.1. Global annotation by differential expression analysis<a id="23"></a>

Seurat identifies markers that define clusters via differential expression. By default, it identifes positive and negative markers of a single cluster (specified in ident.1), compared to all other cells. `FindAllMarkers` automates this process for all clusters, but it is possible to test groups of clusters vs. each other, or against all cells.

The `min.pct` argument requires a feature to be detected at a minimum percentage in either of the two groups of cells, and the `thresh.test` argument requires a feature to be differentially expressed (on average) by some amount between the two groups. You can set both of these to 0, but with a dramatic increase in time - since this will test a large number of features that are unlikely to be highly discriminatory. As another option to speed up these computations, `max.cells.per.ident` can be set. This will downsample each identity class to have no more cells than whatever this is set to. While there is generally going to be a loss in power, the speed increases can be significiant and the most highly differentially expressed features will likely still rise to the top.

In [ ]:
# Seurat default settings
E11.5.markers <- FindAllMarkers(E11_5, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25)

In [ ]:
#dir.create("./6-kmita/analysis")
write.csv(E11.5.markers, file = "./6-kmita/analysis/E11.5.markers.csv", quote = FALSE)

Cluster 0: mesenchymal cells 

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(E11_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(E11_5, c("Prrx1", "Tbx4", "Tbx5", "Prrx2", "Pitx1", "Hoxd9"), min.cutoff = "q1")  

p1 + p2
#ggsave("./6-kmita/images/E11_5_c0.png" ,width=18, height=8)
#prr

Cluster 1: ectoderm

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)

p1 <- DimPlot(E11_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(E11_5, c("Wnt3", "Fgf8", "En1", "Wnt7a"), min.cutoff = "q1") 

p1 + p2

#ggsave("./6-kmita/images/E11_5_c1.png" ,width=16, height=8)

Cluster 2: muscle cells

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)

p1 <- DimPlot(E11_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(E11_5, c("Pax3", "Tnnt1", "Myf5", "Myl1"), min.cutoff = "q1") + xlab("UMAP 1") + ylab("UMAP 2")

p1 + p2

#ggsave("./6-kmita/images/E11_5_c2.png" ,width=16, height=8)

Cluster 3: endothelial cells + bloodcells 

In [ ]:
#Endothelial markers
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(E11_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2") 
p2 <- FeaturePlot(E11_5, c("Pecam1", "Cd34", "Cd93", "Cdh5", "Emcn"), min.cutoff = "q1") 

p1 + p2

#ggsave("./6-kmita/images/E11_5_c3-2.png" ,width=18, height=8)

#Bloodcell markers
options(repr.plot.width = 16, repr.plot.height = 8)

p3 <- DimPlot(E11_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p4 <- FeaturePlot(E11_5, c("Hbb-y", "Hba-x", "Hba-a1", "Hbb-bt"), min.cutoff = "q1")


p3 + p4

#ggsave("./6-kmita/images/E11_5_c3-1.png" ,width=16, height=8)

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)

p3 <- DimPlot(E11_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p4 <- FeaturePlot(E11_5, c("Sox10", "Foxd3", "Fabp7", "Sox2"))

p3 + p4
#ggsave("./6-kmita/images/E11_5_c.png" ,width=12, height=8)
#Check neural makers

**Adding cell types**

In [ ]:
#creating column that contains cell type 
E11_5@meta.data$CellType <- Idents(E11_5)

In [ ]:
E11_5@meta.data$CellType <- as.character(E11_5@meta.data$CellType)

E11_5@meta.data$CellType[E11_5@meta.data$CellType == '0'] <- 'Mesenchyme'
E11_5@meta.data$CellType[E11_5@meta.data$CellType == '1'] <- 'Ectoderm'
E11_5@meta.data$CellType[E11_5@meta.data$CellType == '2'] <- 'Myocytes'
E11_5@meta.data$CellType[E11_5@meta.data$CellType == '3'] <- 'Endothelium + erythrocytes'

E11_5@meta.data$CellType <- as.factor(E11_5@meta.data$CellType)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)

DimPlot(E11_5, reduction = "umap", label = FALSE, group.by = "CellType") + xlab("UMAP 1") + ylab("UMAP 2")
#ggsave("./6-kmita/figures_thesis/E10_5_global_annotated.png" ,width=10, height=8)

In [ ]:
Idents(E11_5) <- E11_5$CellType

In [ ]:
#saveRDS(E11_5, file = "/home/host_home/usb-drive/Thesis/Kmita/6-kmita/RDataSessions/E11_5_GlobalAnnotation_SeuratObject.Rds")

**Visualizing data**

In [ ]:
E11_5@meta.data$CellType_v <- E11_5@meta.data$CellType

seuratObject <- E11_5

seuratObject@meta.data$CellType_v <- as.character(seuratObject@meta.data$CellType_v)

seuratObject@meta.data$CellType_v[seuratObject@meta.data$CellType_v == 'Mesenchyme'] <- '1 - Mesenchyme'
seuratObject@meta.data$CellType_v[seuratObject@meta.data$CellType_v == 'Ectoderm'] <- '2 - Ectoderm'
seuratObject@meta.data$CellType_v[seuratObject@meta.data$CellType_v == 'Myocytes'] <- '3 - Myocytes'
seuratObject@meta.data$CellType_v[seuratObject@meta.data$CellType_v == 'Endothelial cells + erythrocytes'] <- '4 - Endothelial cells + erythrocytes'


seuratObject@meta.data$CellType_v <- as.factor(seuratObject@meta.data$CellType_v)

E11_5 <- seuratObject 

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)

DimPlot(E11_5, reduction = "umap", label = FALSE, group.by = "CellType_v") + xlab("UMAP 1") + ylab("UMAP 2")
#ggsave("./6-kmita/figures_thesis/E11_5_global_annotated.pdf" ,width=10, height=8)

#### 4.4.2. Refinement of global annotations<a id="23.2"></a>
##### 4.4.2.1. Subsetting mesenchymal clusters<a id="24"></a>

In [ ]:
E11_5 <- readRDS("/home/host_home/tom/Kmita/6-kmita/RDataSessions/E11_5_GlobalAnnotation_SeuratObject.Rds")

In [ ]:
Idents(E11_5) <- E11_5@meta.data$CellType

In [ ]:
sc_mesenchymal_E11_5 <- subset(E11_5, idents = c("Mesenchyme"))

In [ ]:
sc_mesenchymal_E11_5

In [ ]:
sc_mesenchymal_E11_5 <- NormalizeData(sc_mesenchymal_E11_5, verbose = FALSE)
sc_mesenchymal_E11_5 <- FindVariableFeatures(sc_mesenchymal_E11_5, selection.method = "vst", nfeatures = 2000)

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(sc_mesenchymal_E11_5), 10)

In [ ]:
sc_mesenchymal_E11_5 <- ScaleData(sc_mesenchymal_E11_5)

In [ ]:
sc_mesenchymal_E11_5 <- RunPCA(sc_mesenchymal_E11_5, features = VariableFeatures(object = sc_mesenchymal_E11_5), verbose = FALSE)

In [ ]:
options(repr.plot.width=10)
DimPlot(sc_mesenchymal_E11_5, reduction = "pca", group.by = "orig.ident") + xlab("PC 1") + ylab("PC 2")

In [ ]:
ElbowPlot(sc_mesenchymal_E11_5, 50)

In [ ]:
sc_mesenchymal_E11_5 <- FindNeighbors(sc_mesenchymal_E11_5, dims = 1:25, verbose = FALSE)
sc_mesenchymal_E11_5 <- FindClusters(sc_mesenchymal_E11_5, resolution = 2, verbose = FALSE)

In [ ]:
sc_mesenchymal_E11_5 <- RunUMAP(sc_mesenchymal_E11_5, dims = 1:25, n.neighbors = 10, verbose = FALSE)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(sc_mesenchymal_E11_5, reduction = "umap", label=TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
#ggsave("./6-kmita/images/E11_5_m_na.png" ,width=10, height=8)

In [ ]:
#sc_mesenchymal <- FindNeighbors(sc_mesenchymal, dims = 1:25, verbose = FALSE)
#sc_mesenchymal <- FindClusters(sc_mesenchymal, resolution = 0.6, verbose = FALSE)

#sc_mesenchymal <- RunUMAP(sc_mesenchymal, dims = 1:25, n.neighbors = 10, verbose = FALSE)

#options(repr.plot.width = 9)
#DimPlot(sc_mesenchymal, reduction = "umap", label=TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

In [ ]:
markers_11_5_mesenchyme <- FindAllMarkers(sc_mesenchymal_E11_5)

In [ ]:
#saveRDS(markers_11_5_mesenchyme, file = "/home/host_home/usb-drive/Thesis/Kmita/6-kmita/analysis/markers_11_5_mesenchyme.Rds")

In [ ]:
markers_11_5_mesenchyme <- readRDS(file = "/home/host_home/usb-drive/Thesis/Kmita/6-kmita/analysis/markers_11_5_mesenchyme.Rds")

In [ ]:
#select only cluster 0 and order avg_log2FC from big to small

#df_m[df_m$cluster == 0,] %>%
#slice_max(n = 5, order_by = avg_log2FC)

**Positional information**

Proximal mesenchyme

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 8)

g <- WhichCells(sc_mesenchymal_E11_5, idents = c("26"))
p1 <- DimPlot(sc_mesenchymal_E11_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")

p2 <- FeaturePlot(sc_mesenchymal_E11_5, c("Shox2" , "Hoxd9", "Hoxa9")) + xlab("UMAP 1") + ylab("UMAP 2")

p1 + p2

In [ ]:
0 6 9 10 14 15 17 23 26

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 8)

g <- WhichCells(sc_mesenchymal_E11_5, idents = c("0","6","9","10","14","15","17","23","26"))
p1 <- DimPlot(sc_mesenchymal_E11_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

p2 <- FeaturePlot(sc_mesenchymal_E11_5, c("Shox2" , "Hoxd9", "Meis1", "Meis2")) + xlab("UMAP 1") + ylab("UMAP 2")

p1 + p2
#ggsave("./6-kmita/images/E11_5_m_proximal.png" ,width=20, height=8)

Intermediate mesenchyme 

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 8)

g <- WhichCells(sc_mesenchymal_E11_5, idents = c("20",'22',"1"))
p1 <- DimPlot(sc_mesenchymal_E11_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")

p2 <- FeaturePlot(sc_mesenchymal_E11_5, c("Hoxa11")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 8)

g <- WhichCells(sc_mesenchymal_E11_5, idents = c("0","6","9","10","14","15","17","23","26","20",'22',"1","21","16"))
p1 <- DimPlot(sc_mesenchymal_E11_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

p2 <- FeaturePlot(sc_mesenchymal_E11_5, c("Hoxa11")) + xlab("UMAP 1") + ylab("UMAP 2")
p3 <- FeaturePlot(sc_mesenchymal_E11_5, c("Shox2" , "Hoxd9", "Meis1", "Meis2")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p3 - p2
options(repr.plot.width = 10, repr.plot.height = 8)
p1

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 8)

g <- WhichCells(sc_mesenchymal_E11_5, idents = c("0","6","9","10","14","23","26","20",'22',"1","21","16"))
p1 <- DimPlot(sc_mesenchymal_E11_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

p2 <- FeaturePlot(sc_mesenchymal_E11_5, c("Hoxa11")) + xlab("UMAP 1") + ylab("UMAP 2")
p3 <- FeaturePlot(sc_mesenchymal_E11_5, c("Shox2" , "Hoxd9", "Meis1", "Meis2")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2
#ggsave("./6-kmita/images/E11_5_m_intermediate.png" ,width=20, height=8)

Distal mesenchyme

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 8)

p1 <- DimPlot(sc_mesenchymal_E11_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E11_5, c("Hoxa13", "Hoxd13")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 9, repr.plot.height = 8)
p1 
options(repr.plot.width = 20, repr.plot.height = 8)
p2

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 8)

g <- WhichCells(sc_mesenchymal_E11_5, idents = c("4","7","8","13","2","3","24","25","24",'19'))
p1 <- DimPlot(sc_mesenchymal_E11_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")

p2 <- FeaturePlot(sc_mesenchymal_E11_5, c("Hoxa13", "Hoxd13")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 10, repr.plot.height = 8)
p1 
options(repr.plot.width = 20, repr.plot.height = 8)
p2

Adding annotation

In [ ]:
sc_mesenchymal_E11_5@meta.data$CellType_PID <- Idents(sc_mesenchymal_E11_5)

In [ ]:
sc_mesenchymal_E11_5@meta.data$CellType_PID <- as.character(sc_mesenchymal_E11_5@meta.data$CellType_PID)

sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '15'] <- 'Proximal cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '17'] <- 'Proximal cells'

sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '0'] <- 'Intermediate cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '1'] <- 'Intermediate cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '10'] <- 'Intermediate cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '11'] <- 'Intermediate cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '12'] <- 'Intermediate cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '14'] <- 'Intermediate cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '16'] <- 'Intermediate cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '18'] <- 'Intermediate cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '20'] <- 'Intermediate cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '21'] <- 'Intermediate cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '22'] <- 'Intermediate cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '23'] <- 'Intermediate cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '26'] <- 'Intermediate cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '27'] <- 'Intermediate cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '5'] <- 'Intermediate cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '6'] <- 'Intermediate cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '9'] <- 'Intermediate cells'

sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '4'] <- 'Distal cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '7'] <- 'Distal cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '8'] <- 'Distal cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '13'] <- 'Distal cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '2'] <- 'Distal cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '3'] <- 'Distal cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '24'] <- 'Distal cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '25'] <- 'Distal cells'
sc_mesenchymal_E11_5@meta.data$CellType_PID[sc_mesenchymal_E11_5@meta.data$CellType_PID == '19'] <- 'Distal cells'

sc_mesenchymal_E11_5@meta.data$CellType_PID <- as.factor(sc_mesenchymal_E11_5@meta.data$CellType_PID)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(sc_mesenchymal_E11_5, label = TRUE, group.by = "CellType_PID")  + xlab("UMAP 1") + ylab("UMAP 2")

**Cell type**

ZPA

In [ ]:
#ZPA

options(repr.plot.width = 20, repr.plot.height = 8)

p1 <- DimPlot(sc_mesenchymal_E11_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E11_5, c("Shh")) + xlab("UMAP 1") + ylab("UMAP 2")

p1 + p2

#ggsave("./6-kmita/images/E11_5_m_ZPA.png" ,width=20, height=8)

Cartilage precursors

In [ ]:
#Finding distal cartilage cells
g <- WhichCells(sc_mesenchymal_E11_5, idents = c("7","23","20","5","14","15","3","16"))
p1 <- DimPlot(sc_mesenchymal_E11_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

p2 <- FeaturePlot(sc_mesenchymal_E11_5, c("Sox9", "Prrx1")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 10, repr.plot.height = 8)
p1 
options(repr.plot.width = 20, repr.plot.height = 8)
p2
#ggsave("./6-kmita/images/E11_5_m_cartilage-1.png" ,width=10, height=8, plot=p1)
#ggsave("./6-kmita/images/E11_5_m_cartilage-2.png" ,width=20, height=8, plot=p2)

Tenocytes

In [ ]:
g <- WhichCells(sc_mesenchymal_E11_5, idents = c("11"))
p1 <- DimPlot(sc_mesenchymal_E11_5, cells.highlight = list(g)) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E11_5, c("Scx", "Dcn", "Tnc", "Mkx")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

In [ ]:
g <- WhichCells(sc_mesenchymal_E11_5, idents = c("6","21","18","27",'0'))
p1 <- DimPlot(sc_mesenchymal_E11_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E11_5, c("Scx", "Dcn", "Tnc", "Mkx")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

#ggsave("./6-kmita/images/E11_5_m_tenocytes.png" ,width=20, height=8)

Joint precursors

In [ ]:
g <- WhichCells(sc_mesenchymal_E11_5, idents = c("17"))
p1 <- DimPlot(sc_mesenchymal_E11_5, cells.highlight = list(g), label = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_mesenchymal_E11_5, c("Gdf5", "Wnt9", "Smoc2", "Dact2")) + xlab("UMAP 1") + ylab("UMAP 2")

options(repr.plot.width = 20, repr.plot.height = 8)
p1 + p2

#ggsave("./6-kmita/images/E11_5_m_joint.png" ,width=20, height=8)

In [ ]:
options(repr.plot.width = 28, repr.plot.height = 8)
FeaturePlot(sc_mesenchymal_E11_5, c("Gdf5", "Sox9"), blend = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

#ggsave("./6-kmita/images/E11_5_Gdf5_Sox9.png" ,width=28, height=8)

###### Checking annotation

In [ ]:
A <- sc_mesenchymal_E11_5

In [ ]:
# Paraxial Mesoderm: Tbx6+, Sox2-
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Tbx6", "Sox2"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Intermediate Mesoderm: Osr1, Wt1, Pax2
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Osr1", "Wt1", "Pax2"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Lateral plate mesoderm
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Foxf1", "Bmp4", "Hoxb6", "Hand1", "Hand2", "Gata4","Prrx1"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Early limb bud mesenchyme
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Meis1", "Tbx4", "Tbx5"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# ZPA
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Shh"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Chondroprogenitors
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = FALSE) + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Prrx1", "Prrx2", "Sox9"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Resting zone chondrocytes
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Fgfr3", "Pcp4", "Pthlh", "Sfrp5", "Spon1"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Proliferative chondrocytes
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Sox9", "Mki67", "Top2a", "Pth1r"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Pre-hypertrophic chondrocytes
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Ihh", "Ppa1", "Smpd3"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# SSPC
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Lepr","Pdgfrb",'Prrx1',"Eng", "Nt5e", "Thy1","Cxcl12","Sp7","Adipoq"), min.cutoff = "q1") 

p1 + p2

In [ ]:
# SSPC
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Lepr","Pdgfrb",'Prrx1',"Eng", "Nt5e", "Thy1","Cxcl12","Sp7","Adipoq"), min.cutoff = "q1") 

p1 + p2

In [ ]:
# Osteoprogenitors
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Runx2", "Sp7"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Osteoblasts
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Col1a1", "Bglap"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Osteocytes
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Dmp1"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Osteoclasts
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Csf1", "Ctsk"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Periosteal stem cells
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Pdpn", "Nt5e", "Cd164"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Periosteal progenitors
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Col5a2", "Twist2", "Tbx15"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Periosteum
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Postn"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Joint precursors: sox9-
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Gdf5", "Wnt9a", "Smoc2", "Dact2","Sox9"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Interzone cells: sox9-
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Smoc2", "Dact2", "Wnt4", "Wnt16","Sox9"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Tenocyte precursors
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE, group.by="CellType") + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Tnmd", "Tppp3", "Scx", "Prrx1", "Prrx2"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Tenocytes
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Scx", "Tnmd"), min.cutoff = "q1")  

p1 + p2

In [ ]:
# Synovial fibroblasts: Hla-
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(A, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(A, c("Dkk1", "Dra","Hla"), min.cutoff = "q1")  

p1 + p2

##### Adding annotation

In [ ]:
sc_mesenchymal_E11_5@meta.data$CellType_g <- Idents(sc_mesenchymal_E11_5)

In [ ]:
sc_mesenchymal_E11_5@meta.data$CellType_g <- as.character(sc_mesenchymal_E11_5@meta.data$CellType_g)

sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '7'] <- 'Chondroprogenitors'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '23'] <- 'Chondroprogenitors'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '20'] <- 'Chondroprogenitors'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '5'] <- 'Chondroprogenitors'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '14'] <- 'Chondroprogenitors'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '15'] <- 'Chondroprogenitors'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '3'] <- 'Chondroprogenitors'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '16'] <- 'Chondroprogenitors'

sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '17'] <- 'Joint precursors'

sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '6'] <- 'Tenocyte precursors'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '21'] <- 'Tenocyte precursors'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '18'] <- 'Tenocyte precursors'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '27'] <- 'Tenocyte precursors'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '0'] <- 'Tenocyte precursors'

sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '13'] <- 'ZPA'

sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '1'] <- 'Limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '11'] <- 'Limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '10'] <- 'Limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '12'] <- 'Limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '18'] <- 'Limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '19'] <- 'Limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '2'] <- 'Limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '22'] <- 'Limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '24'] <- 'Limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '25'] <- 'Limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '26'] <- 'Limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '4'] <- 'Limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '8'] <- 'Limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType_g[sc_mesenchymal_E11_5@meta.data$CellType_g == '9'] <- 'Limb bud mesenchyme'

sc_mesenchymal_E11_5@meta.data$CellType_g <- as.factor(sc_mesenchymal_E11_5@meta.data$CellType_g)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)
DimPlot(sc_mesenchymal_E11_5, label = FALSE, group.by = 'CellType_g') + xlab("UMAP 1") + ylab("UMAP 2")

#ggsave("./6-kmita/images/E11_5_m_tenocytes.png" ,width=12, height=8)

##### Adding annotation - final

In [ ]:
sc_mesenchymal_E11_5@meta.data$CellType <- Idents(sc_mesenchymal_E11_5)

In [ ]:
sc_mesenchymal_E11_5@meta.data$CellType <- as.character(sc_mesenchymal_E11_5@meta.data$CellType)

sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '7'] <- 'Chondroprogenitors'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '23'] <- 'Chondroprogenitors'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '20'] <- 'Chondroprogenitors'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '5'] <- 'Chondroprogenitors'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '14'] <- 'Chondroprogenitors'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '15'] <- 'Chondroprogenitors'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '3'] <- 'Chondroprogenitors'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '16'] <- 'Chondroprogenitors'

sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '17'] <- 'Joint precursors'

sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '1'] <- 'Intermediate limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '11'] <- 'Intermediate limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '10'] <- 'Intermediate limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '12'] <- 'Intermediate limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '18'] <- 'Intermediate limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '22'] <- 'Intermediate limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '26'] <- 'Intermediate limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '9'] <- 'Intermediate limb bud mesenchyme'

sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '19'] <- 'Distal limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '2'] <- 'Distal limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '24'] <- 'Distal limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '25'] <- 'Distal limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '4'] <- 'Distal limb bud mesenchyme'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '8'] <- 'Distal limb bud mesenchyme'

sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '6'] <- 'Tenocyte precursors'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '21'] <- 'Tenocyte precursors'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '18'] <- 'Tenocyte precursors'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '27'] <- 'Tenocyte precursors'
sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '0'] <- 'Tenocyte precursors'

sc_mesenchymal_E11_5@meta.data$CellType[sc_mesenchymal_E11_5@meta.data$CellType == '13'] <- 'ZPA'

sc_mesenchymal_E11_5@meta.data$CellType <- as.factor(sc_mesenchymal_E11_5@meta.data$CellType)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)
DimPlot(sc_mesenchymal_E11_5, label = FALSE, group.by = 'CellType') + xlab("UMAP 1") + ylab("UMAP 2")

#ggsave("./6-kmita/images/E11_5_m_tenocytes.png" ,width=12, height=8)

In [ ]:
saveRDS(sc_mesenchymal_E11_5, file ="/home/host_home/tom/Kmita/6-kmita/RDataSessions/sc_mesenchymal_E11_5_clustered_SeuratObject.Rds")

##### 4.4.2.1. Subsetting ectodermal cluster<a id="25"></a>

In [ ]:
E11_5 <- readRDS("/home/jovyan/researcher_home/tom/Kmita/6-kmita/RDataSessions/E11_5_GlobalAnnotation_SeuratObject.Rds")

In [ ]:
Idents(E11_5) <- E11_5@meta.data$CellType

In [ ]:
sc_ectoderm_E11_5 <- subset(E11_5, idents = c("Ectoderm"))

In [ ]:
sc_ectoderm_E11_5 <- NormalizeData(sc_ectoderm_E11_5, verbose = FALSE)
sc_ectoderm_E11_5 <- FindVariableFeatures(sc_ectoderm_E11_5, selection.method = "vst", nfeatures = 2000)

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(sc_ectoderm_E11_5), 10)

In [ ]:
sc_ectoderm_E11_5 <- ScaleData(sc_ectoderm_E11_5)

In [ ]:
sc_ectoderm_E11_5 <- RunPCA(sc_ectoderm_E11_5, features = VariableFeatures(object = sc_ectoderm_E11_5), verbose = FALSE)

In [ ]:
options(repr.plot.width = 9)

DimPlot(sc_ectoderm_E11_5, reduction = "pca", group.by = "orig.ident") + xlab("PC 1") + ylab("PC 2")

In [ ]:
ElbowPlot(sc_ectoderm_E11_5, 50)

In [ ]:
sc_ectoderm_E11_5 <- FindNeighbors(sc_ectoderm_E11_5, dims = 1:40)
sc_ectoderm_E11_5 <- FindClusters(sc_ectoderm_E11_5, resolution = 2)

In [ ]:
sc_ectoderm_E11_5 <- RunUMAP(sc_ectoderm_E11_5, dims = 1:40, n.neighbors = 10, verbose = FALSE)

In [ ]:
options(repr.plot.width=8)
DimPlot(sc_ectoderm_E11_5, reduction = "umap", label=TRUE, group.by = "seurat_clusters")

In [ ]:
df <- FindAllMarkers(sc_ectoderm_E11_5)

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive
# ones
#df %>%
#    group_by(cluster) %>%
#    slice_max(n = 10, order_by = avg_log2FC) 

In [ ]:
#select only cluster 0 and order avg_log2FC from big to small

df[df$cluster == 6,] %>%
slice_max(n = 20, order_by = avg_log2FC)

In [ ]:
#df['Meis2',]

AER

In [ ]:
options(repr.plot.width = 20)

p1 <- DimPlot(sc_ectoderm_E11_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_ectoderm_E11_5, c("Fgf8", "Fgf4")) + xlab("UMAP 1") + ylab("UMAP 2")

p1 + p2

#ggsave("./6-kmita/images/E11_5_e_AER.png" ,width=20, height=8)

Neurons

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)

p1 <- DimPlot(sc_ectoderm_E11_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_ectoderm_E11_5, c("Nefl", "Mapt")) + xlab("UMAP 1") + ylab("UMAP 2")

p1 + p2
#ggsave("./6-kmita/images/E11_5_e_neurons.png" ,width=20, height=8)

Skin

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)

p1 <- DimPlot(sc_ectoderm_E11_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_ectoderm_E11_5, c("Krt5", "Krt10","Krt7")) + xlab("UMAP 1") + ylab("UMAP 2")

p1 + p2
#ggsave("./6-kmita/images/E11_5_e_neurons.png" ,width=20, height=8)

Skin

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)

p1 <- DimPlot(sc_ectoderm_E11_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_ectoderm_E11_5, c("Wnt7a", "En1")) + xlab("UMAP 1") + ylab("UMAP 2")

p1 + p2
#ggsave("./6-kmita/images/E11_5_e_neurons.png" ,width=20, height=8)

Adding annotation

In [ ]:
sc_ectoderm_E11_5@meta.data$CellType <- Idents(sc_ectoderm_E11_5)

In [ ]:
sc_ectoderm_E11_5@meta.data$CellType <- as.character(sc_ectoderm_E11_5@meta.data$CellType)

sc_ectoderm_E11_5@meta.data$CellType[sc_ectoderm_E11_5@meta.data$CellType == '0'] <- 'AER'
sc_ectoderm_E11_5@meta.data$CellType[sc_ectoderm_E11_5@meta.data$CellType == '1'] <- 'Ectoderm'
sc_ectoderm_E11_5@meta.data$CellType[sc_ectoderm_E11_5@meta.data$CellType == '2'] <- 'Ectoderm'
sc_ectoderm_E11_5@meta.data$CellType[sc_ectoderm_E11_5@meta.data$CellType == '3'] <- 'Ectoderm'
sc_ectoderm_E11_5@meta.data$CellType[sc_ectoderm_E11_5@meta.data$CellType == '4'] <- 'Ectoderm'
sc_ectoderm_E11_5@meta.data$CellType[sc_ectoderm_E11_5@meta.data$CellType == '5'] <- 'Neurons'
sc_ectoderm_E11_5@meta.data$CellType[sc_ectoderm_E11_5@meta.data$CellType == '6'] <- 'Keratinocytes'
sc_ectoderm_E11_5@meta.data$CellType[sc_ectoderm_E11_5@meta.data$CellType == '7'] <- 'Ectoderm'
sc_ectoderm_E11_5@meta.data$CellType[sc_ectoderm_E11_5@meta.data$CellType == '8'] <- 'Ectoderm'

sc_ectoderm_E11_5@meta.data$CellType <- as.factor(sc_ectoderm_E11_5@meta.data$CellType)

In [ ]:
options(repr.plot.width=8)
DimPlot(sc_ectoderm_E11_5, reduction = "umap", label=TRUE, group.by = "CellType")

In [ ]:
saveRDS(sc_ectoderm_E11_5, file = "/home/jovyan/researcher_home/tom/Kmita/6-kmita/RDataSessions/sc_ectoderm_E11_5_annotated_SeuratObject.Rds")

##### 4.4.2.2. Subsetting blood and endothelial cluster<a id="25-2"></a>

In [ ]:
E11_5 <- readRDS("/home/host_home/tom/Kmita/6-kmita/RDataSessions/E11_5_GlobalAnnotation_SeuratObject.Rds")

In [ ]:
Idents(E11_5) <- E11_5@meta.data$CellType

In [ ]:
sc_BE_E11_5 <- subset(E11_5, idents = c("Endothelium + erythrocytes"))

In [ ]:
sc_BE_E11_5 <- NormalizeData(sc_BE_E11_5, verbose = FALSE)
sc_BE_E11_5 <- FindVariableFeatures(sc_BE_E11_5, selection.method = "vst", nfeatures = 2000)

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(sc_BE_E11_5), 10)

In [ ]:
sc_BE_E11_5 <- ScaleData(sc_BE_E11_5)

In [ ]:
sc_BE_E11_5 <- RunPCA(sc_BE_E11_5, features = VariableFeatures(object = sc_BE_E11_5), verbose = FALSE)

In [ ]:
options(repr.plot.width = 9)

DimPlot(sc_BE_E11_5, reduction = "pca", group.by = "orig.ident") + xlab("PC 1") + ylab("PC 2")

In [ ]:
ElbowPlot(sc_BE_E11_5, 50)

In [ ]:
sc_BE_E11_5 <- FindNeighbors(sc_BE_E11_5, dims = 1:22)
sc_BE_E11_5 <- FindClusters(sc_BE_E11_5, resolution = 1)

In [ ]:
sc_BE_E11_5 <- RunUMAP(sc_BE_E11_5, dims = 1:22, n.neighbors = 10, verbose = FALSE)

In [ ]:
options(repr.plot.width=8)
DimPlot(sc_BE_E11_5, reduction = "umap", label=TRUE, group.by = "seurat_clusters")

In [ ]:
df <- FindAllMarkers(sc_BE_E11_5)

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive
# ones
#df %>%
#    group_by(cluster) %>%
#    slice_max(n = 10, order_by = avg_log2FC) 

In [ ]:
#select only cluster 0 and order avg_log2FC from big to small

df[df$cluster == 1,] %>%
slice_max(n = 10, order_by = avg_log2FC)

In [ ]:
#df['Meis2',]

Endothilium

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(sc_BE_E11_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2") 
p2 <- FeaturePlot(sc_BE_E11_5, c("Vcam1", "Cd34", "Cd93", "Cdh5","Pecam1"), min.cutoff = "q1") 

p1 + p2

#ggsave("./6-kmita/images/E11_5_be_endo.png" ,width=18, height=12)

Erythrocytes

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)

p1 <- DimPlot(sc_BE_E11_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_BE_E11_5, c("Cd235a", "Gypa"), min.cutoff = "q1")


p1 + p2
#ggsave("./6-kmita/images/E11_5_be_rbc.png" ,width=16, height=8)

Macrophages

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)

p1 <- DimPlot(sc_BE_E11_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_BE_E11_5, c("Cd3g", "Cd19", "Cd68", "Cxcr2"), min.cutoff = "q1")


p1 + p2
#ggsave("./6-kmita/images/E11_5_be_mx.png" ,width=16, height=8)

Adding annotation

In [ ]:
sc_BE_E11_5@meta.data$CellType <- Idents(sc_BE_E11_5)

In [ ]:
sc_BE_E11_5@meta.data$CellType <- as.character(sc_BE_E11_5@meta.data$CellType)

sc_BE_E11_5@meta.data$CellType[sc_BE_E11_5@meta.data$CellType == '1'] <- 'Immune cells'
sc_BE_E11_5@meta.data$CellType[sc_BE_E11_5@meta.data$CellType == '2'] <- 'Vascular endothelial cells'
sc_BE_E11_5@meta.data$CellType[sc_BE_E11_5@meta.data$CellType == '0'] <- 'Erythroid cells'

sc_BE_E11_5@meta.data$CellType <- as.factor(sc_BE_E11_5@meta.data$CellType)

In [ ]:
options(repr.plot.width=8)
DimPlot(sc_BE_E11_5, reduction = "umap", label=TRUE, group.by = "CellType")

In [ ]:
saveRDS(sc_BE_E11_5, file = "/home/host_home/tom/Kmita/6-kmita/RDataSessions/sc_BE_E11_5_annotated_SeuratObject.Rds")

##### 4.4.2.3. Subsetting myocytes

In [ ]:
E11_5 <- readRDS("/home/host_home/tom/Kmita/6-kmita/RDataSessions/E11_5_GlobalAnnotation_SeuratObject.Rds")

In [ ]:
Idents(E11_5) <- E11_5@meta.data$CellType

In [ ]:
sc_myocytes_E11_5 <- subset(E11_5, idents = c("Myocytes"))

In [ ]:
sc_myocytes_E11_5 <- NormalizeData(sc_myocytes_E11_5, verbose = FALSE)
sc_myocytes_E11_5 <- FindVariableFeatures(sc_myocytes_E11_5, selection.method = "vst", nfeatures = 2000)

# Identify the 10 most highly variable genes
top10 <- head(VariableFeatures(sc_myocytes_E11_5), 10)

In [ ]:
sc_myocytes_E11_5 <- ScaleData(sc_myocytes_E11_5)

In [ ]:
sc_myocytes_E11_5 <- RunPCA(sc_myocytes_E11_5, features = VariableFeatures(object = sc_myocytes_E11_5), verbose = FALSE)

In [ ]:
options(repr.plot.width = 9)

DimPlot(sc_myocytes_E11_5, reduction = "pca", group.by = "orig.ident") + xlab("PC 1") + ylab("PC 2")

In [ ]:
ElbowPlot(sc_myocytes_E11_5, 50)

In [ ]:
sc_myocytes_E11_5 <- FindNeighbors(sc_myocytes_E11_5, dims = 1:20)
sc_myocytes_E11_5 <- FindClusters(sc_myocytes_E11_5, resolution = 1)

In [ ]:
sc_myocytes_E11_5 <- RunUMAP(sc_myocytes_E11_5, dims = 1:20, n.neighbors = 10, verbose = FALSE)

In [ ]:
options(repr.plot.width=8)
DimPlot(sc_myocytes_E11_5, reduction = "umap", label=TRUE, group.by = "seurat_clusters")

In [ ]:
df <- FindAllMarkers(sc_BE_E11_5)

In [ ]:
# find markers for every cluster compared to all remaining cells, report only the positive
# ones
#df %>%
#    group_by(cluster) %>%
#    slice_max(n = 10, order_by = avg_log2FC) 

In [ ]:
#select only cluster 0 and order avg_log2FC from big to small

df[df$cluster == 1,] %>%
slice_max(n = 10, order_by = avg_log2FC)

In [ ]:
#df['Meis2',]

Muscle progenitors

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 12)

p1 <- DimPlot(sc_myocytes_E11_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2") 
p2 <- FeaturePlot(sc_myocytes_E11_5, c("Pax7", "Notch", "Itm2a", "Myf5", "Pax3", "Myod1"), min.cutoff = "q1") 

p1 + p2

#ggsave("./6-kmita/images/E11_5_be_endo.png" ,width=18, height=12)

Myogenic stem cells

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)

p1 <- DimPlot(sc_myocytes_E11_5, label = TRUE) + NoLegend() + xlab("UMAP 1") + ylab("UMAP 2")
p2 <- FeaturePlot(sc_myocytes_E11_5, c("Pax7","Myf5"), min.cutoff = "q1")


p1 + p2
#ggsave("./6-kmita/images/E11_5_be_rbc.png" ,width=16, height=8)

Adding annotation

In [ ]:
sc_myocytes_E11_5@meta.data$CellType <- Idents(sc_myocytes_E11_5)

In [ ]:
sc_myocytes_E11_5@meta.data$CellType <- as.character(sc_myocytes_E11_5@meta.data$CellType)

sc_myocytes_E11_5@meta.data$CellType[sc_myocytes_E11_5@meta.data$CellType == '2'] <- 'Muscle progenitors'

sc_myocytes_E11_5@meta.data$CellType[sc_myocytes_E11_5@meta.data$CellType == '3'] <- 'Myogenic stem cells'
sc_myocytes_E11_5@meta.data$CellType[sc_myocytes_E11_5@meta.data$CellType == '0'] <- 'Myogenic stem cells'
sc_myocytes_E11_5@meta.data$CellType[sc_myocytes_E11_5@meta.data$CellType == '1'] <- 'Myogenic stem cells'

sc_myocytes_E11_5@meta.data$CellType <- as.factor(sc_myocytes_E11_5@meta.data$CellType)

In [ ]:
options(repr.plot.width=8)
DimPlot(sc_myocytes_E11_5, reduction = "umap", label=TRUE, group.by = "CellType")

In [ ]:
saveRDS(sc_myocytes_E11_5, file = "/home/host_home/tom/Kmita/6-kmita/RDataSessions/sc_myocytes_E11_5_annotated_SeuratObject.Rds")

#### 3.4.2. Updating annotations <a id="16"></a>

In [ ]:
E11_5 <- readRDS("/home/jovyan/researcher_home/tom/Kmita/6-kmita/RDataSessions/E11_5_GlobalAnnotation_SeuratObject.Rds")

In [ ]:
sc_mesenchymal_E11_5 <- readRDS(file = "/home/jovyan/researcher_home/tom/Kmita/6-kmita/RDataSessions/sc_mesenchymal_E11_5_clustered_SeuratObject.Rds")

In [ ]:
sc_ectoderm_E11_5 <- readRDS(file = "/home/jovyan/researcher_home/tom/Kmita/6-kmita/RDataSessions/sc_ectoderm_E11_5_annotated_SeuratObject.Rds")

In [ ]:
sc_BE_E11_5 <- readRDS(file = "/home/jovyan/researcher_home/tom/Kmita/6-kmita/RDataSessions/sc_BE_E11_5_annotated_SeuratObject.Rds")

In [ ]:
sc_myocytes_E11_5 <- readRDS(file = "/home/jovyan/researcher_home/tom/Kmita/6-kmita/RDataSessions/sc_myocytes_E11_5_annotated_SeuratObject.Rds")

In [ ]:
meta1 <- E11_5@meta.data
meta2 <- sc_mesenchymal_E11_5@meta.data

In [ ]:
df <- split(meta1, meta1$CellType)

In [ ]:
mesenchyme <- rownames(df$Mesenchyme)

In [ ]:
mesenchyme_meta <- meta2[mesenchyme, c("CellType", "orig.ident")]
mesenchyme_meta$orig.ident <- NULL

head(mesenchyme_meta)

In [ ]:
df$Mesenchyme <- cbind(df$Mesenchyme, mesenchyme_meta)
#df$mesenchyme$CellType <- df$mesenchyme$CellType
#df$mesenchyme$labels_mesenchyme_E11_5 <- NULL

In [ ]:
str(df)

In [ ]:
ectoderm <- rownames(df$Ectoderm)

In [ ]:
meta2 <- sc_ectoderm_E11_5@meta.data
ectoderm_meta <- meta2[ectoderm, c("CellType", "orig.ident")]
ectoderm_meta$orig.ident <- NULL

head(ectoderm_meta)

In [ ]:
df$Ectoderm <- cbind(df$Ectoderm, ectoderm_meta)

In [ ]:
BE <- rownames(df$`Endothelium + erythrocytes`)

In [ ]:
meta2 <- sc_BE_E11_5@meta.data
BE_meta <- meta2[BE, c("CellType", "orig.ident")]
BE_meta$orig.ident <- NULL

head(BE_meta)

In [ ]:
df$`Endothelium + erythrocytes` <- cbind(df$'Endothelium + erythrocytes', BE_meta)

In [ ]:
myocytes <- rownames(df$Myocytes)

In [ ]:
meta2 <- sc_myocytes_E11_5@meta.data
myocytes_meta <- meta2[myocytes, c("CellType", "orig.ident")]
myocytes_meta$orig.ident <- NULL

head(myocytes_meta)

In [ ]:
df$Myocytes <- cbind(df$Myocytes, myocytes_meta)

In [ ]:
df$Mesenchyme$Barcode = rownames(df$Mesenchyme)
df$Ectoderm$Barcode = rownames(df$Ectoderm)
df$`Endothelium + erythrocytes`$Barcode = rownames(df$`Endothelium + erythrocytes`)
df$Myocytes$Barcode = rownames(df$Myocytes)

**Make sure all dataframes in the list have the same number of columns, in the same order.**

In [ ]:
str(df)

In [ ]:
names(df)

In [ ]:
meta.data <- do.call(rbind, df)
rownames(meta.data) = meta.data$Barcode
meta.data$Barcode <- NULL

In [ ]:
head(meta.data)

In [ ]:
dim(E11_5@meta.data)
dim(meta.data)

In [ ]:
meta.data$CellType <- NULL

In [ ]:
dim(E11_5@meta.data)
dim(meta.data)

In [ ]:
target <- rownames(E11_5@meta.data)
meta.data <- meta.data[match(target, rownames(meta.data)),]

In [ ]:
E11_5@meta.data = meta.data

#### 5.4.3. Finalized embedding and annotation<a id="37"></a>

In [ ]:
options(repr.plot.width=10)
DimPlot(E11_5, group.by = "CellType", label = FALSE, repel = TRUE) + xlab("UMAP 1") + ylab("UMAP 2")

#ggsave("./6-kmita/images/E11_5_a.png" ,width=10, height=8)

In [ ]:
Idents(E11_5) <- E11_5$CellType

In [ ]:
saveRDS(E11_5, file = "/home/jovyan/researcher_home/tom/Kmita/6-kmita/RDataSessions/E11_5_seuratObject_annotation.Rds")

In [ ]:
E11_5